# Order-free matching performance

In [142]:
# Generic imports

import glob
import os
import pandas as pd
from os import listdir
from os.path import isfile, join
import ast
import json


# Sklearn imports
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, auc, roc_curve

### Read order-free candidates

In [127]:
picos = 'I'

In [128]:
order_free_doc_file = f'/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/order_free/doc/{picos}/train_ebm_intervention_syn.json'
orf_doc = dict()

with open( order_free_doc_file, 'r' ) as rf:
    orf_doc = json.load(rf)

In [156]:
def get_orf(orf, partial: bool):
        
    orf_doc_offsets = dict()

    for i, (k, v) in enumerate(orf.items()):

        if i == 0:
            continue

        if 'Inters. (full)' in v and len(v['Inters. (full)']) > 0 and len(v['tokens']) > 1:

            for k_i, v_i in v['Inters. (full)'].items():

                orf_doc_offsets[k_i] = []

                if len( v_i['tokens'] ) > 1:

                    source_tokens = v_i['tokens']
                    source_offsets = v_i['char offs.']
                    orf_doc_offsets[k_i].extend( source_offsets )
                    
        if partial == True:
            
            if 'Inters. (partial)' in v and len(v['Inters. (partial)']) > 0 and len(v['tokens']) > 1:

                for k_i, v_i in v['Inters. (partial)'].items():

                    orf_doc_offsets[k_i] = []

                    if len( v_i['tokens'] ) > 1:

                        source_tokens = v_i['tokens']
                        source_offsets = v_i['char offs.']
                        orf_doc_offsets[k_i].extend( source_offsets )
            
    
    return orf_doc_offsets

In [158]:
orf_doc_offsets_partial = get_orf(orf, partial = True)
orf_doc_offsets_full = get_orf(orf, partial = False)

In [159]:
len(orf_doc_offsets_partial)

4216

In [160]:
len(orf_doc_offsets_full)

2440

### Load order-bound matches

In [163]:
def order_free_matches(x, orf_offsets):
    
    labs_modified = []
    
    for i, (identifier, offs, labs) in enumerate( zip(x.pmid, x.offsets, x.labels) ):
             
        lab_val = [v for k, v in ast.literal_eval(labs).items()] 
        off_val = ast.literal_eval(offs) 
        
        if str(identifier) in orf_offsets: 
            orf_matches =  orf_offsets[ str(identifier) ]
            match_indices = [ off_val.index(m) for m in orf_matches ]
            for i, l in enumerate(lab_val):
                if i in match_indices:
                    lab_val[i] = 1
                    
        labs_modified.append( lab_val )
        
        
    return labs_modified

In [135]:
order_bound_file = f'/mnt/nas2/results/Results/systematicReview/order_free_matching/EBM_PICO_training_matches/direct/{picos}/lf_ds_intervention_syn.tsv'

In [136]:
data = pd.read_csv(order_bound_file, sep='\t', header=0)

In [138]:
# Fetch ground truth

# coarse
picos_coarse_ = data[str(ent.lower())]
picos_coarse = [ l for ent_series in picos_coarse_ for l in ast.literal_eval(ent_series) ]

# fine
picos_fine_ = data[ str(ent.lower()) + '_f' ]
picos_fine = [ l for ent_series in picos_fine_ for l in ast.literal_eval(ent_series) ]
picos_fine = ['1' if (n != '1' and n != '0') else str(n) for i, n in enumerate(picos_fine) ]

In [137]:
# Fetch order-bound predictions

preds = [ v for ent_series in data['labels'] for k, v in ast.literal_eval(ent_series).items() ]
order_bound_preds = ['0' if n == -1 else str(n) for i, n in enumerate(preds) ]

In [167]:
# Fetch order-free predictions

# ORF with partial
data['orf_doc_labels_partial'] = order_free_matches( data, orf_doc_offsets_partial )
preds_orf_doc_partial = [ v for ent_series in data['orf_doc_labels_partial'] for v in ent_series ]
order_free_doc_partial_preds = ['0' if n == -1 else str(n) for i, n in enumerate(preds_orf_doc_partial) ]

# ORF with full
data['orf_doc_labels_full'] = order_free_matches( data, orf_doc_offsets_full )
preds_orf_doc_full = [ v for ent_series in data['orf_doc_labels_full'] for v in ent_series ]
order_free_doc_full_preds = ['0' if n == -1 else str(n) for i, n in enumerate(preds_orf_doc_full) ]

### evaluation metrics

##### direct matching

In [144]:
# Classification report
cr_order_bound_coarse = classification_report( picos_coarse, order_bound_preds, digits=4  )
print('Confusion matrix for coarse-grained ground truth and order-bound matches')
print( cr_order_bound_coarse )

cr_order_bound_fine = classification_report( picos_fine, order_bound_preds, digits=4  )
print('\n\nConfusion matrix for fine-grained ground truth and order-bound matches')
print( cr_order_bound_fine )

Confusion matrix for coarse-grained ground truth and order-bound matches
              precision    recall  f1-score   support

           0     0.8836    0.8093    0.8448   1147447
           1     0.1323    0.2143    0.1636    155722

    accuracy                         0.7382   1303169
   macro avg     0.5080    0.5118    0.5042   1303169
weighted avg     0.7938    0.7382    0.7634   1303169



Confusion matrix for fine-grained ground truth and order-bound matches
              precision    recall  f1-score   support

           0     0.9129    0.8108    0.8588   1183318
           1     0.1122    0.2362    0.1522    119851

    accuracy                         0.7579   1303169
   macro avg     0.5126    0.5235    0.5055   1303169
weighted avg     0.8393    0.7579    0.7938   1303169



In [165]:
# Classification report
cr_order_free_doc_full_coarse = classification_report( picos_coarse, order_free_doc_full_preds, digits=4  )
print('Confusion matrix for coarse-grained ground truth and document level order-free matches')
print( cr_order_free_doc_full_coarse )

cr_order_free_doc_full_fine = classification_report( picos_fine, order_free_doc_full_preds, digits=4  )
print('\n\nConfusion matrix for fine-grained ground truth and document level order-free matches')
print( cr_order_free_doc_full_fine )

Confusion matrix for coarse-grained ground truth and document level order-free matches
              precision    recall  f1-score   support

           0     0.8837    0.8089    0.8446   1147447
           1     0.1329    0.2159    0.1645    155722

    accuracy                         0.7380   1303169
   macro avg     0.5083    0.5124    0.5046   1303169
weighted avg     0.7940    0.7380    0.7634   1303169



Confusion matrix for fine-grained ground truth and document level order-free matches
              precision    recall  f1-score   support

           0     0.9130    0.8103    0.8586   1183318
           1     0.1127    0.2379    0.1530    119851

    accuracy                         0.7577   1303169
   macro avg     0.5129    0.5241    0.5058   1303169
weighted avg     0.8394    0.7577    0.7937   1303169



In [168]:
# Classification report
cr_order_free_doc_partial_coarse = classification_report( picos_coarse, order_free_doc_partial_preds, digits=4  )
print('Confusion matrix for coarse-grained ground truth and document level order-free (partial) matches')
print( cr_order_free_doc_partial_coarse )

cr_order_free_doc_partial_fine = classification_report( picos_fine, order_free_doc_partial_preds, digits=4  )
print('\n\nConfusion matrix for fine-grained ground truth and document level order-free (partial) matches')
print( cr_order_free_doc_partial_fine )

Confusion matrix for coarse-grained ground truth and document level order-free (partial) matches
              precision    recall  f1-score   support

           0     0.8838    0.8064    0.8433   1147447
           1     0.1330    0.2188    0.1654    155722

    accuracy                         0.7362   1303169
   macro avg     0.5084    0.5126    0.5044   1303169
weighted avg     0.7941    0.7362    0.7623   1303169



Confusion matrix for fine-grained ground truth and document level order-free (partial) matches
              precision    recall  f1-score   support

           0     0.9132    0.8079    0.8573   1183318
           1     0.1129    0.2414    0.1539    119851

    accuracy                         0.7558   1303169
   macro avg     0.5130    0.5246    0.5056   1303169
weighted avg     0.8396    0.7558    0.7926   1303169

